<a href="https://colab.research.google.com/github/123jyoti614/Trading_Strategy_Prediction/blob/main/closing_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd

# creating a data frame
df1 = pd.read_csv("XBTUSD-1h-data.csv")
df = pd.DataFrame(df1)

df['close'] = pd.to_numeric(df['close'], errors='coerce').astype(float)
print(f"df['close'] dtype after conversion: {df['close'].dtype}")
print(f"df['close'] head after conversion:\n{df['close'].head()}")

df['timestamp'] = pd.to_datetime(df['timestamp']).dt.date

print(df.groupby('timestamp')['close'].mean())

df['close'] dtype after conversion: float64
df['close'] head after conversion:
0    237.45
1    236.08
2    236.34
3    235.71
4    235.75
Name: close, dtype: float64
timestamp
2015-09-25      235.780000
2015-09-26      234.295000
2015-09-27      233.854167
2015-09-28      237.647826
2015-09-29      238.405000
                  ...     
2026-01-03    90003.204167
2026-01-04    91215.745833
2026-01-05    93123.525000
2026-01-06    93275.400000
2026-01-07    92571.466667
Name: close, Length: 3758, dtype: float64


In [15]:
def price_matrix_creator(data, seq_len=30):
    '''
    It converts the series into a nested list where every item of the list contains historic prices of 30 days
    '''
    price_matrix = []
    for index in range(len(data)-seq_len+1):
        price_matrix.append(data[index:index+seq_len])
    return price_matrix

def normalize_windows(window_data):
    '''
    It normalizes each value to reflect the percentage changes from starting point
    '''
    normalised_data = []
    for window in window_data:
        normalised_window = [((float(p) / float(window.iloc[0])) - 1) for p in window]
        normalised_data.append(normalised_window)
    return normalised_data

def train_test_split_(price_matrix, train_size=0.9, shuffle=False, return_row=True):
    '''
    It makes a custom train test split where the last part is kept as the training set.
    '''
    price_matrix = np.array(price_matrix)
    #print(price_matrix.shape)
    row = int(round(train_size * len(price_matrix)))
    train = price_matrix[:row, :]
    if shuffle==True:
        np.random.shuffle(train)
    X_train, y_train = train[:row,:-1], train[:row,-1]
    X_test, y_test = price_matrix[row:,:-1], price_matrix[row:,-1]
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    if return_row:
        return row, X_train, y_train, X_test, y_test
    else:
        X_train, y_train, X_test, y_test

In [16]:
ser = hist_price_dl() # Not passing any argument since they are set by default
price_matrix = price_matrix_creator(ser) # Creating a matrix using the dataframe
price_matrix = normalize_windows(price_matrix) # Normalizing its values to fit to RNN
row, X_train, y_train, X_test, y_test = train_test_split_(price_matrix) # Applying train-test splitting, also returning the splitting-point



In [17]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
import time
# LSTM Model parameters, I chose
batch_size = 2            # Batch size (you may try different values)
epochs = 15               # Epoch (you may try different values)
seq_len = 30            # 30 sequence data (Representing the last 30 days)
loss='mean_squared_error' # Since the metric is MSE/RMSE
optimizer = 'rmsprop'     # Recommended optimizer for RNN
activation = 'linear'     # Linear activation
input_shape=(None,1)      # Input dimension
output_dim = 30           # Output dimension

In [18]:
model = Sequential()
model.add(LSTM(units=output_dim, return_sequences=True, input_shape=input_shape))
model.add(Dense(units=32,activation=activation))
model.add(LSTM(units=output_dim, return_sequences=False))
model.add(Dense(units=1,activation=activation))
model.compile(optimizer=optimizer,loss=loss)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [19]:

start_time = time.time()
model.fit(x=X_train,
          y=y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.05)
end_time = time.time()
processing_time = end_time - start_time

Epoch 1/15
38534/38534 ━━━━━━━━━━━━━━━━━━━━ 500s 13ms/step - loss: 1.7945e-04 - val_loss: 5.8032e-05
Epoch 2/15
38534/38534 ━━━━━━━━━━━━━━━━━━━━ 489s 13ms/step - loss: 8.1907e-05 - val_loss: 4.3477e-05
Epoch 3/15
38534/38534 ━━━━━━━━━━━━━━━━━━━━ 494s 13ms/step - loss: 7.7825e-05 - val_loss: 3.5020e-05
Epoch 4/15
38534/38534 ━━━━━━━━━━━━━━━━━━━━ 499s 13ms/step - loss: 7.7799e-05 - val_loss: 4.2194e-05
Epoch 5/15
38534/38534 ━━━━━━━━━━━━━━━━━━━━ 487s 12ms/step - loss: 7.5613e-05 - val_loss: 3.9490e-05
Epoch 6/15
38534/38534 ━━━━━━━━━━━━━━━━━━━━ 489s 13ms/step - loss: 7.7421e-05 - val_loss: 3.6533e-05
Epoch 7/15
38534/38534 ━━━━━━━━━━━━━━━━━━━━ 492s 13ms/step - loss: 7.2331e-05 - val_loss: 3.8022e-05
Epoch 8/15
38534/38534 ━━━━━━━━━━━━━━━━━━━━ 521s 14ms/step - loss: 7.4261e-05 - val_loss: 4.5219e-05
Epoch 9/15
38534/38534 ━━━━━━━━━━━━━━━━━━━━ 534s 13ms/step - loss: 7.2579e-05 - val_loss: 3.5074e-05
Epoch 10/15
38534/38534 ━━━━━━━━━━━━━━━━━━━━ 495s 13ms/step - loss: 7.3716e-05 - val_loss: 

In [20]:

model.save('coin_predictor.h5')

In [22]:
import requests,json,numpy as np,pandas as pd
#We need ser, preds, row
ser = hist_price_dl()[1:31]
price_matrix = price_matrix_creator(ser)
X_test = normalize_windows(price_matrix)
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [23]:

from keras.models import load_model
model = load_model('coin_predictor.h5')
preds = model.predict(X_test, batch_size=2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 593ms/step


In [29]:
def deserializer(preds, data, train_size=0.9, train_phase=False):
    '''
    Arguments:
    preds : Predictions to be converted back to their original values
    data : It takes the data into account because the normalization was made based on the full historic data
    train_size : Only applicable when used in train_phase
    train_phase : When a train-test split is made, this should be set to True so that a cut point (row) is calculated based on the train_size argument, otherwise cut point is set to 0

    Returns:
    A list of deserialized prediction values, original true values, and date values for plotting
    '''
    price_matrix = np.array(price_matrix_creator(ser)) # Still using global 'ser'
    if train_phase:
        row = int(round(train_size * len(price_matrix)))
    else:
        row=0

    # Correctly extract datetime.date objects using integer indices from df['timestamp']
    date_indices = ser.index[row+29:]
    date = df['timestamp'].loc[date_indices]
    date = np.reshape(date, (date.shape[0]))

    X_test = price_matrix[row:,:-1]
    y_test = price_matrix[row:,-1]
    preds_original = []
    preds = np.reshape(preds, (preds.shape[0]))
    for index in range(0, len(preds)):
        pred = (preds[index]+1)* X_test[index][0] # Note: X_test might be empty if ser is short
        preds_original.append(pred)
    preds_original = np.array(preds_original)

    if train_phase:
        return [date, y_test, preds_original]
    else:
        import datetime
        # Perform element-wise addition to handle datetime.date objects
        date_plus_one_day = np.array([d + datetime.timedelta(days=1) for d in date])
        return [date_plus_one_day, y_test]

In [30]:

final_pred = deserializer(preds, ser, train_size=0.9, train_phase=False)
final_pred[1][0]

np.float64(234.0)